# Derivação da rede MLP (backpropagation)

![title](mlp-imagem.jpg)

## Variáveis

- $X$: matriz vertical $(n_0 \times 1)$ com os valores de entrada da rede neural para uma determinada amostrada;


- $W_1$, $W_2$ e $W_3$: matrizes de pesos entre as camadas neurais (vide figura) com dimensões $(\text{neurônios de saída} \times \text{neurônios/valores de entrada})$ de modo que suas colunas são preenchidos com os valores dos pesos conectados a cada entrada da rede neural:
$
\begin{bmatrix}
x1_1 & x2_1 & x3_1\\ 
x1_2 & x2_2 & x3_2
\end{bmatrix}
$


- $I_1$, $I_2$ e $I_3$: matriz de entrada ponderada nas camadas neurais;


- $Y_1$, $Y_2$ e $Y_3$: matriz de saída das camadas neurais após aplicação da função de ativação;


- $d$: matriz dos valores de saída $(m \times 1)$ desejados da rede neural.

## Função de ativação

Deve ser contínua e diferenciável em todo seu domínio. 

- **Sigmóide**:
$\frac{1}{1+e^{-x}}$

 - vantagens: pequena variação do gradiente, saídas entre 0 e 1 normalizando a saída cada neurônio, previsões claras à medida que se eleva o valor X tendo y os valores 0 ou 1.
 - desvantagens: Vanishing gradient (quando o gradiente fica tão pequeno que praticamente não altera os valores da matriz de pesos), saídas não centralizadas no zero, sem valores negativos. 
 
 
- **Tangente hiperbólica**:
$\tanh(x)$
    - vantagens: mesmas da sigmóide com a vantagem de ter valores negativos, zero e positivos para y
    - desvantagens: praticamente as mesmas da sigmóide
    
    
- **ReLu**:
$\max(0, x)$
    - vantagens: permite desativar certos neurônios por ter derivada zero para valores negativos de x. 
    - desvantagens: próximo do zero ou valores negativos há o problema da aprendizagem por backpropagation
    
    
- **Leaky ReLU**:
$f(x) = ax, x < 0$,
$f(x) = x, x \geqslant 0$
    - vantagens: resolve o problema de aprendizagem nos valores negativos e próximos a zero do ReLu
    - desvantagens: pode haver valores inconsistentes quando x < 0.
    obs: geralmente $a=0.01$


- **Swish**:
$\frac{x}{1+e^{-x}}$
    - vantagens: performance melhor que ReLu com mesmo esforço computacional

## Cálculo do erro

### Matriz erro na camada de saída
$\text{erro} = d - Y_3 = \begin{bmatrix}
e_1\\ 
e_2\\ 
...
\end{bmatrix}$

Com a matriz $\text{erro}$ de dimensões $(m \times 1)$ uma vez que há $m$ neurônios na saída da rede.

### Parâmetro do erro de desempenho local
Para uma determinada amostra:

$
E = \frac{e_1^2+e_2^2 + ... + e_m^2}{2}$

em que $e_m$ é o erro da $m$-ésima saída da rede neural. 

### Erro quadrático médio
Considerando todas as $p$ amostras, o erro quadrático médio é dado por

$E_M = \frac{(E_1 + E_2 + ... + E_p)}{p}$


## Implementação do algoritmo forward - MLP

1) Para uma dada amostra de valores da matriz de entrada $X$, tem-se que os vetores de entrada da primeira camada neural é dado por:

- $I_1 = W_1 \times X$ 

Dimensão: $(n_1 \times 1)$

2) Após isso, encontra-se a matriz da saída dos neurônios $(Y_1)$ com relação à camada 1, em que $f(x)$ representa o valor da função de ativação do neurônio:

- $Y_1 = f(I_1)$

Dimensão: $(n_1 \times 1)$

3) Para a entrada dos neurônios da segunda camada têm-se:

- $I_2 = W_2 \times Y_1$

Dimensão: $(n_2 \times 1)$

4) E as saídas da segunda camada:

- $Y_2 = f(I_2)$

Dimensão: $(n_2 \times 1)$

5) Analogamente para terceira camada:

- $I_3 = W_3 \times Y_2$

Dimensão: $(m \times 1)$

- $Y_3 = f(I_3)$

Dimensão: $(m \times 1)$

6) Cálculo do erro:

- $\text{erro} = d - Y_3$

- erro de desempenho local: $E = \frac{e_1^2+e_2^2 + ... + e_m^2}{2}$

## Implementação do algoritmo backforward - MLP

### Para camada de saída (3):

$\nabla E_3 = \frac{\partial E}{\partial W_3} = \frac{\partial E}{\partial Y_3}.\frac{\partial Y_3}{\partial I_3}.\frac{\partial I_3}{\partial W_3}$

- $\frac{\partial E}{\partial Y_3}$ (desempenho local)$ = \frac{2}{2}(d - Y_3)(-1) = -(d - Y_3)$

- $\frac{\partial Y_3}{\partial I_3} = f'(I_3)$

- $\frac{\partial I_3}{\partial W_3} = Y_2^T$

Assim, 

$\nabla E_3 = (-(d - Y_3)f'(I_3)) \times Y_2^T$

Como o gradiente tem direção crescente e queremos minimizar o erro, o ajuste de pesos da matrix $W_3$ tem que ser na direção oposta do gradiente de modo que:

$\Delta W_3 = -(-(d - Y_3)f'(I_3)) \times Y_2^T = \eta \delta_3 \times Y_2^T$

em que $\eta$ é a taxa de aprendizagem e $\delta_3 = (d - Y_3)f'(I_3)$ com dimensão $(m \times 1)$

Finalmente,

$W_{3final} = W_{3inicial} + \eta \delta_3 \times Y_2^T$

### Para camada escondida (2):

$\nabla E_2 = \frac{\partial E}{\partial W_2} = \frac{\partial E}{\partial Y_2}.\frac{\partial Y_2}{\partial I_2}.\frac{\partial I_2}{\partial W_2}$

 - $\frac{\partial E}{\partial Y_2} = \frac{\partial E}{\partial I_3} . \frac{\partial I_3}{\partial Y_2} = W_3^T \times \frac{\partial E}{\partial I_3} = W_3^T \times (-(d - Y_3).f'(I_3)) = -W_3^T \times \delta_3$
 - $\frac{\partial Y_2}{\partial I_2} = f'(I_2)$
 - $\frac{\partial I_2}{\partial W_2} = Y_1^T$
 
Então, 

$-\nabla E_2 = -\frac{\partial E}{\partial W_2} = -(-W_3^T \times \delta_3 . f'(I_2)) \times Y_1^T$

Definindo $\delta_2 = (W_3^T \times \delta_3). f'(I_2)$ com dimensão $(n_2 \times 1)$, tem-se, analogamente, para a matriz de peso $W_2$:

$W_{2final} = W_{2inicial} + \eta \delta_2 \times Y_1^T$

### Para camada inicial (1):

De forma análoga às camadas anteriores, tem-se:

$\delta_1 = (W_2^T \times \delta_2). f'(I_1)$ com dimensão $(n_1 \times 1)$

$W_{1final} = W_{1inicial} + \eta \delta_1 \times X^T$

#### OBSERVAÇÃO: As matrizes de pesos devem ser alteradas no mesmo instante, então, $\delta_1$ é em função do valor antigo de $W_2$ uma vez que tem que ser calculado ANTES que a atualização das matrizes de peso. 

## Generalização das equações

Sendo $L$ a $L$-ésima camada neural com $W_L$ é a matriz de pesos entre as camadas $L$ e $L+1$, as equações de correção dos pesos é generalizado por:

$W_{L_{final}} = W_{L_{inicial}} + \eta . \delta_L \times Y_{L-1}^T$

para $L \geqslant 1$ com $Y_0 = X$ tal que

$\delta_L = (W_{L+1}^T \times \delta_{L+1}).f'(I_L)$

sendo satisfeito para $L < \text{camada de saída}$, caso contrário

$\delta_L = (d - Y_L).f'(I_L)$

## Critério de parada da interação

A interação termina quando a diferença entre os erros quadráticos médios de duas épocas sucessivas for menor ou igual à precisão desejada:

$|E_{M_{atual}} - E_{M_{anterior}}| \leqslant \varepsilon$

ou então, se preferir, quando o erro quadrático médio atinge um valor desejado.

## Importância do bias

Em todo livro de redes neurais, considera-se um *bias*, ou seja, um valor, geralmente unitário, que é conectado as entradas das camadas neurais a partir de um peso. Sua função consiste em deslocar (*shift*) o gráfico resultante da função de ativação escolhido. 

Para exemplificar, seja uma rede neural com apenas uma entrada e um neurônio (sem bias) com função de ativação $y=x$, sua saída será dada por:

$y = f(xw) = xw$

Como pode-se perceber, se trata de uma reta que corta a origem no primeiro e terceiro quadrante (para $w>0$) e no segundo e quarto (para $w<0$). Para os valores de $x=1,2,3$ e $d=1,2,3$, respectivamente, poderíamos treinar a rede sem um bias sem maiores problemas, mas quando $x=1,2,3$ e $d=5,4,3$ ?

Nesta situação, é preciso que haja um deslocamento no gráfico, no eixo $y$, para satisfazer a equação $y = -d + 6$ e é exatamente nesse ponto que o bias se torna importante. No primeiro exemplo (tanh(x)) fiz um teste sem bias e foi possível convergir a rede, já no exemplo 2 usando a função de ativação *leaky ReLu* não obtive o mesmo sucesso, obtendo erros elevados na ordem de $7.77$. Realizando o mesmo teste com o bias, a *leaky ReLu* teve um desempenho até melhor convergindo rapidamente. Por esse motivo, é importante que sempre seja adicionado o bias nas entradas das camadas neurais e atualizado seu valor como mostrado no exemplo 3. 

# Algoritmo em Python

In [5]:
# Mostrar todos os output's
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Exemplo 1 - tanh(x)

In [7]:
import numpy as np
import pandas as pd


# Base de dados a ser treinada
# Entradas
x = pd.DataFrame(
	[[1],
	[2],
	[3]],
	columns=['valores x'])

# Valor desejado para saída
d = pd.DataFrame(
	[[5],
	[4],
	[3]],
	columns=['valores desejados'])

# Convertendo o dataframe em array e normalizando os valores desejados (para ficar entre 0 e +1)
x = x.to_numpy()
d = d/(1.05*d.max())
d = d.to_numpy()

# Derivada de tanh(x) = sech²(x) = 1 - (tanh(x))²
def df(x):
    y = 1 - np.power(np.tanh(x), 2)
    return y

#def rede_mlp(n, x, d, net, k, precisao):

# *** Construindo a rede de duas camadas ***

# número de neurônios na primeira camada
net=3
# taxa de aprendizagem
n = 0.1
# precisão do erro quadrático médio
precisao=0.01

# Inicializar matrizes de pesos com valores aleatórios distintos
w1 = np.random.rand(net,len(x[0]))
w2 = np.random.rand(1,net)

# Erro quadrático médio inicial e número de épocas
E_M=1
epocas=0

while E_M>precisao:
    E_M=0
    errofinal=0
    for i in range(0,len(x)):
        
        # FOWARD
        i1 = np.matmul(w1, x[i].reshape(len(x[i]),1))
        y1 = np.tanh(i1)

        i2 = np.matmul(w2, y1)
        y2 = np.tanh(i2)
        
        # erro com o valor desejado
        erro = d[i].reshape(len(d[i]),1) - y2
        
        # BACKPROPAGATION
        delta_2 = erro*df(i2)
        delta_1 = (np.matmul(w2.T, delta_2))*df(i1)
        
        w2 = w2 + n*(np.matmul(delta_2, y1.reshape(1, net)))
        w1 = w1 + n*(np.matmul(delta_1, x[i].reshape(1, len(x[i]))))
        
        errofinal = errofinal + 0.5*erro**2
        
    E_M = errofinal/len(x)
    epocas+=1
    print(E_M)
    
    
def conferir_peso(w1, w2, entrada, fator_multiplicativo):
    i1 = np.matmul(w1, np.array(entrada).T)
    y1 = np.tanh(i1)
    i2 = np.matmul(w2, y1)
    y2 = np.tanh(i2)
    y2 = y2*fator_multiplicativo
    return(y2)

# Conferir o número de épocas para a convergência e o valor encontrado para alguma das entradas
print('\n')
valor = conferir_peso(w1, w2, [2], 1.05*5)
print(valor)
print(epocas)


[[0.03538428]]
[[0.0352496]]
[[0.03512102]]
[[0.0349979]]
[[0.03487971]]
[[0.03476593]]
[[0.03465612]]
[[0.03454989]]
[[0.03444688]]
[[0.03434675]]
[[0.03424923]]
[[0.03415405]]
[[0.03406096]]
[[0.03396974]]
[[0.0338802]]
[[0.03379215]]
[[0.03370542]]
[[0.03361986]]
[[0.03353533]]
[[0.0334517]]
[[0.03336885]]
[[0.03328666]]
[[0.03320503]]
[[0.03312387]]
[[0.03304309]]
[[0.03296261]]
[[0.03288234]]
[[0.03280222]]
[[0.03272219]]
[[0.03264217]]
[[0.03256212]]
[[0.03248197]]
[[0.03240168]]
[[0.0323212]]
[[0.03224048]]
[[0.03215949]]
[[0.03207819]]
[[0.03199653]]
[[0.0319145]]
[[0.03183205]]
[[0.03174916]]
[[0.0316658]]
[[0.03158195]]
[[0.03149758]]
[[0.03141269]]
[[0.03132724]]
[[0.03124122]]
[[0.03115462]]
[[0.03106742]]
[[0.03097963]]
[[0.03089121]]
[[0.03080218]]
[[0.03071252]]
[[0.03062224]]
[[0.03053132]]
[[0.03043978]]
[[0.03034761]]
[[0.03025481]]
[[0.0301614]]
[[0.03006738]]
[[0.02997276]]
[[0.02987755]]
[[0.02978177]]
[[0.02968542]]
[[0.02958854]]
[[0.02949112]]
[[0.0293932]]
[[0.

## Exemplo 2 - Leaky ReLu (sem bias)

In [9]:
import numpy as np
import pandas as pd


# Base de dados a ser treinada
x = pd.DataFrame(
	[[1],
	[2],
	[3]],
	columns=['valores x'])

d = pd.DataFrame(
	[[5],
	[4],
	[3]],
	columns=['valores desejados'])

# Convertendo o dataframe em array e normalizando os valores desejados para ficar entre 0 e +1.
x = x.to_numpy()
# d = d/(1.05*d.max())
d = d.to_numpy()


# Derivada de tanh(x) = sech²(x) = 1 - (tanh(x))²
def df(x):
    x = np.array(x)
    x[x<=0] = 0.01
    x[x>0] = 1
    return x

def f(x):
    return(np.where(x > 0, x, x * 0.01))
#     return(np.maximum(x, 0))

    

#def rede_mlp(n, x, d, net, k, precisao):

# Construindo a rede de duas camadas 
# net = número de neurônios na primeira camada
# n = taxa de aprendizagem
# precisao = precisão do erro quadrático médio
net=3
n = 1e-4
precisao=0.0001
w1 = np.random.rand(net,len(x[0]))
w2 = np.random.rand(1,net)
E_M=20
epocas=0

while E_M>precisao:
    E_M=0
    errofinal=0
    for i in range(0,len(x)):
        
        # FOWARD
        i1 = np.matmul(w1, x[i].reshape(len(x[i]),1))
        y1 = f(i1)
        
        

        i2 = np.matmul(w2, y1)
        y2 = f(i2)
        
        
        # erro com o valor desejado
        erro = d[i].reshape(len(d[i]),1) - y2
        
        
        # BACKPROPAGATION
        delta_2 = erro*df(i2)
        delta_1 = (np.matmul(w2.T, delta_2))*df(i1)
        
        w2 = w2 + n*(np.matmul(delta_2, y1.reshape(1, net)))
        w1 = w1 + n*(np.matmul(delta_1, x[i].reshape(1, len(x[i]))))
        
          
        errofinal = errofinal + 0.5*erro**2
        
    #E_M = errofinal/len(x)
    E_M = errofinal
    epocas+=1
    print(E_M)
    
    
def conferir_peso(w1, w2, entrada, fator_multiplicativo):
    i1 = np.matmul(w1, np.array(entrada).T)
    y1 = np.tanh(i1)

    i2 = np.matmul(w2, y1)
    y2 = np.tanh(i2)
    y2 = y2*fator_multiplicativo
    return(y2)

#1.05*5
valor = conferir_peso(w1, w2, [2], 1)
valor
print(epocas)

[[10.06134831]]
[[10.04655047]]
[[10.03182541]]
[[10.01717296]]
[[10.00259295]]
[[9.98808521]]
[[9.97364956]]
[[9.95928583]]
[[9.94499385]]
[[9.93077344]]
[[9.91662443]]
[[9.90254663]]
[[9.88853987]]
[[9.87460396]]
[[9.86073873]]
[[9.84694399]]
[[9.83321955]]
[[9.81956524]]
[[9.80598087]]
[[9.79246625]]
[[9.77902119]]
[[9.7656455]]
[[9.752339]]
[[9.73910149]]
[[9.72593278]]
[[9.71283268]]
[[9.699801]]
[[9.68683753]]
[[9.67394209]]
[[9.66111448]]
[[9.64835451]]
[[9.63566196]]
[[9.62303665]]
[[9.61047837]]
[[9.59798693]]
[[9.58556212]]
[[9.57320374]]
[[9.56091159]]
[[9.54868547]]
[[9.53652516]]
[[9.52443047]]
[[9.51240118]]
[[9.5004371]]
[[9.48853801]]
[[9.4767037]]
[[9.46493398]]
[[9.45322862]]
[[9.44158742]]
[[9.43001017]]
[[9.41849665]]
[[9.40704666]]
[[9.39565998]]
[[9.3843364]]
[[9.3730757]]
[[9.36187767]]
[[9.3507421]]
[[9.33966877]]
[[9.32865747]]
[[9.31770797]]
[[9.30682007]]
[[9.29599354]]
[[9.28522817]]
[[9.27452374]]
[[9.26388003]]
[[9.25329682]]
[[9.2427739]]
[[9.23231103]]
[

[[7.72750823]]
[[7.72745061]]
[[7.7273935]]
[[7.7273369]]
[[7.72728081]]
[[7.72722523]]
[[7.72717015]]
[[7.72711557]]
[[7.72706147]]
[[7.72700787]]
[[7.72695474]]
[[7.7269021]]
[[7.72684993]]
[[7.72679822]]
[[7.72674699]]
[[7.72669621]]
[[7.7266459]]
[[7.72659603]]
[[7.72654662]]
[[7.72649765]]
[[7.72644912]]
[[7.72640103]]
[[7.72635338]]
[[7.72630615]]
[[7.72625935]]
[[7.72621297]]
[[7.72616701]]
[[7.72612146]]
[[7.72607633]]
[[7.7260316]]
[[7.72598727]]
[[7.72594335]]
[[7.72589982]]
[[7.72585668]]
[[7.72581393]]
[[7.72577157]]
[[7.72572959]]
[[7.72568799]]
[[7.72564677]]
[[7.72560592]]
[[7.72556543]]
[[7.72552531]]
[[7.72548556]]
[[7.72544616]]
[[7.72540712]]
[[7.72536843]]
[[7.72533009]]
[[7.7252921]]
[[7.72525444]]
[[7.72521713]]
[[7.72518016]]
[[7.72514352]]
[[7.72510721]]
[[7.72507123]]
[[7.72503558]]
[[7.72500025]]
[[7.72496523]]
[[7.72493054]]
[[7.72489615]]
[[7.72486208]]
[[7.72482831]]
[[7.72479486]]
[[7.7247617]]
[[7.72472884]]
[[7.72469628]]
[[7.72466402]]
[[7.72463204]]
[[

[[7.7218195]]
[[7.72181945]]
[[7.7218194]]
[[7.72181935]]
[[7.7218193]]
[[7.72181925]]
[[7.7218192]]
[[7.72181916]]
[[7.72181911]]
[[7.72181906]]
[[7.72181901]]
[[7.72181897]]
[[7.72181892]]
[[7.72181888]]
[[7.72181883]]
[[7.72181879]]
[[7.72181874]]
[[7.7218187]]
[[7.72181865]]
[[7.72181861]]
[[7.72181857]]
[[7.72181852]]
[[7.72181848]]
[[7.72181844]]
[[7.7218184]]
[[7.72181836]]
[[7.72181832]]
[[7.72181828]]
[[7.72181824]]
[[7.7218182]]
[[7.72181816]]
[[7.72181812]]
[[7.72181808]]
[[7.72181804]]
[[7.721818]]
[[7.72181796]]
[[7.72181793]]
[[7.72181789]]
[[7.72181785]]
[[7.72181782]]
[[7.72181778]]
[[7.72181774]]
[[7.72181771]]
[[7.72181767]]
[[7.72181764]]
[[7.7218176]]
[[7.72181757]]
[[7.72181753]]
[[7.7218175]]
[[7.72181747]]
[[7.72181743]]
[[7.7218174]]
[[7.72181737]]
[[7.72181733]]
[[7.7218173]]
[[7.72181727]]
[[7.72181724]]
[[7.72181721]]
[[7.72181718]]
[[7.72181715]]
[[7.72181711]]
[[7.72181708]]
[[7.72181705]]
[[7.72181702]]
[[7.72181699]]
[[7.72181697]]
[[7.72181694]]
[[7.7218

[[7.72256636]]
[[7.72256636]]
[[7.72256636]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256637]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.72256638]]
[[7.722566

[[7.72256662]]
[[7.72256662]]
[[7.72256662]]
[[7.72256662]]
[[7.72256662]]
[[7.72256662]]
[[7.72256662]]
[[7.72256662]]
[[7.72256662]]
[[7.72256662]]
[[7.72256662]]
[[7.72256662]]
[[7.72256662]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256663]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256664]]
[[7.72256665]]
[[7.72256665]]
[[7.72256665]]
[[7.72256665]]
[[7.72256665]]
[[7.72256665]]
[[7.722566

...

[[7.84767839]]
[[7.84768657]]
[[7.84769476]]
[[7.84770294]]
[[7.84771113]]
[[7.84771932]]
[[7.8477275]]
[[7.8477357]]
[[7.84774389]]
[[7.84775208]]
[[7.84776028]]
[[7.84776848]]
[[7.84777667]]
[[7.84778487]]
[[7.84779308]]
[[7.84780128]]
[[7.84780948]]
[[7.84781769]]
[[7.8478259]]
[[7.84783411]]
[[7.84784232]]
[[7.84785053]]
[[7.84785875]]
[[7.84786696]]
[[7.84787518]]
[[7.8478834]]
[[7.84789162]]
[[7.84789984]]
[[7.84790806]]
[[7.84791629]]
[[7.84792451]]
[[7.84793274]]
[[7.84794097]]
[[7.8479492]]
[[7.84795743]]
[[7.84796567]]
[[7.8479739]]
[[7.84798214]]
[[7.84799038]]
[[7.84799862]]
[[7.84800686]]
[[7.8480151]]
[[7.84802335]]
[[7.84803159]]
[[7.84803984]]
[[7.84804809]]
[[7.84805634]]
[[7.84806459]]
[[7.84807285]]
[[7.8480811]]
[[7.84808936]]
[[7.84809762]]
[[7.84810588]]
[[7.84811414]]
[[7.8481224]]
[[7.84813067]]
[[7.84813893]]
[[7.8481472]]
[[7.84815547]]
[[7.84816374]]
[[7.84817201]]
[[7.84818029]]
[[7.84818856]]
[[7.84819684]]
[[7.84820512]]
[[7.8482134]]
[[7.84822168]]
[[7.84

[[7.86827657]]
[[7.86828905]]
[[7.86830153]]
[[7.86831402]]
[[7.86832651]]
[[7.868339]]
[[7.86835149]]
[[7.86836398]]
[[7.86837648]]
[[7.86838898]]
[[7.86840149]]
[[7.86841399]]
[[7.8684265]]
[[7.86843901]]
[[7.86845152]]
[[7.86846404]]
[[7.86847656]]
[[7.86848908]]
[[7.8685016]]
[[7.86851413]]
[[7.86852666]]
[[7.86853919]]
[[7.86855173]]
[[7.86856426]]
[[7.8685768]]
[[7.86858935]]
[[7.86860189]]
[[7.86861444]]
[[7.86862699]]
[[7.86863954]]
[[7.8686521]]
[[7.86866466]]
[[7.86867722]]
[[7.86868978]]
[[7.86870235]]
[[7.86871492]]
[[7.86872749]]
[[7.86874006]]
[[7.86875264]]
[[7.86876522]]
[[7.8687778]]
[[7.86879038]]
[[7.86880297]]
[[7.86881556]]
[[7.86882815]]
[[7.86884075]]
[[7.86885334]]
[[7.86886594]]
[[7.86887855]]
[[7.86889115]]
[[7.86890376]]
[[7.86891637]]
[[7.86892898]]
[[7.8689416]]
[[7.86895422]]
[[7.86896684]]
[[7.86897946]]
[[7.86899209]]
[[7.86900472]]
[[7.86901735]]
[[7.86902998]]
[[7.86904262]]
[[7.86905526]]
[[7.8690679]]
[[7.86908055]]
[[7.86909319]]
[[7.86910584]]
[[7.

[[7.89411712]]
[[7.89413643]]
[[7.89415574]]
[[7.89417503]]
[[7.89419432]]
[[7.89421361]]
[[7.89423289]]
[[7.89425217]]
[[7.89427144]]
[[7.89429072]]
[[7.89431]]
[[7.89432927]]
[[7.89434855]]
[[7.89436783]]
[[7.89438711]]
[[7.89440639]]
[[7.89442568]]
[[7.89444497]]
[[7.89446426]]
[[7.89448355]]
[[7.89450285]]
[[7.89452214]]
[[7.89454145]]
[[7.89456075]]
[[7.89458007]]
[[7.89459938]]
[[7.8946187]]
[[7.89463802]]
[[7.89465735]]
[[7.89467668]]
[[7.89469601]]
[[7.89471535]]
[[7.89473469]]
[[7.89475404]]
[[7.89477339]]
[[7.89479275]]
[[7.89481211]]
[[7.89483147]]
[[7.89485084]]
[[7.89487021]]
[[7.89488959]]
[[7.89490897]]
[[7.89492836]]
[[7.89494775]]
[[7.89496715]]
[[7.89498655]]
[[7.89500595]]
[[7.89502536]]
[[7.89504478]]
[[7.8950642]]
[[7.89508362]]
[[7.89510305]]
[[7.89512248]]
[[7.89514192]]
[[7.89516136]]
[[7.8951808]]
[[7.89520025]]
[[7.89521971]]
[[7.89523917]]
[[7.89525863]]
[[7.8952781]]
[[7.89529757]]
[[7.89531705]]
[[7.89533653]]
[[7.89535602]]
[[7.89537551]]
[[7.89539501]]
[[

[[7.91107408]]
[[7.91109713]]
[[7.91112019]]
[[7.91114325]]
[[7.91116632]]
[[7.91118939]]
[[7.91121247]]
[[7.91123555]]
[[7.91125864]]
[[7.91128174]]
[[7.91130483]]
[[7.91132794]]
[[7.91135105]]
[[7.91137416]]
[[7.91139728]]
[[7.9114204]]
[[7.91144353]]
[[7.91146666]]
[[7.9114898]]
[[7.91151295]]
[[7.91153609]]
[[7.91155925]]
[[7.91158241]]
[[7.91160557]]
[[7.91162874]]
[[7.91165192]]
[[7.91167509]]
[[7.91169828]]
[[7.91172147]]
[[7.91174466]]
[[7.91176786]]
[[7.91179107]]
[[7.91181428]]
[[7.91183749]]
[[7.91186071]]
[[7.91188394]]
[[7.91190717]]
[[7.9119304]]
[[7.91195364]]
[[7.91197689]]
[[7.91200014]]
[[7.91202339]]
[[7.91204665]]
[[7.91206992]]
[[7.91209319]]
[[7.91211647]]
[[7.91213975]]
[[7.91216303]]
[[7.91218633]]
[[7.91220962]]
[[7.91223292]]
[[7.91225623]]
[[7.91227954]]
[[7.91230286]]
[[7.91232618]]
[[7.91234951]]
[[7.91237284]]
[[7.91239617]]
[[7.91241952]]
[[7.91244286]]
[[7.91246622]]
[[7.91248957]]
[[7.91251294]]
[[7.9125363]]
[[7.91255968]]
[[7.91258305]]
[[7.91260644]]

KeyboardInterrupt: 

## Exemplo 3 - Leaky ReLu (com bias)

In [1]:
import numpy as np
import pandas as pd


# Base de dados a ser treinada
x = pd.DataFrame(
    [[1],
    [2],
    [3]],
    columns=['valores x'])

d = pd.DataFrame(
    [[5],
    [4],
    [3]],
    columns=['valores desejados'])

# Convertendo o dataframe em array e normalizando os valores desejados para ficar entre 0 e +1.
x = x.to_numpy()
d = d.to_numpy()


def df(x):
    return(np.where(np.array(x) <= 0, 0.01, 1))

def f(x):
    return(np.where(np.array(x) > 0, x, x * 0.01))


#def rede_mlp(n, x, d, net, k, precisao):

# Construindo a rede de duas camadas 
# net = número de neurônios na primeira camada
# n = taxa de aprendizagem
# precisao = precisão do erro quadrático médio
net=3
n = 1e-3
precisao=1e-5
w1 = np.random.rand(net,len(x[0]))
bias1 = np.random.rand()

w2 = np.random.rand(1,net)
bias2 = np.random.rand()

E_M=20
epocas=0

while E_M>precisao:
    E_M=0
    errofinal=0
    for i in range(0,len(x)):

        # FOWARD
        i1 = np.matmul(w1, x[i].reshape(len(x[i]),1)) + bias1
        y1 = f(i1)


        i2 = np.matmul(w2, y1) + bias2
        y2 = f(i2)


        # erro com o valor desejado
        erro = d[i].reshape(len(d[i]),1) - y2


        # BACKPROPAGATION
        delta_2 = erro*df(i2)
        delta_1 = (np.matmul(w2.T, delta_2))*df(i1)
        bias2 += n * delta_2
        bias1 += n * delta_1
        
        w2 = w2 + n*(np.matmul(delta_2, y1.reshape(1, net)))
        w1 = w1 + n*(np.matmul(delta_1, x[i].reshape(1, len(x[i]))))


        errofinal = errofinal + 0.5*erro**2

    #E_M = errofinal/len(x)
    E_M = errofinal
    epocas+=1
    print(E_M)

[[7.61484429]]
[[7.49293547]]
[[7.37750728]]
[[7.26829137]]
[[7.16502003]]
[[7.06742731]]
[[6.97525005]]
[[6.88822881]]
[[6.80610874]]
[[6.72864032]]
[[6.65558007]]
[[6.58669115]]
[[6.52174379]]
[[6.46051578]]
[[6.40279277]]
[[6.3483685]]
[[6.29704503]]
[[6.24863281]]
[[6.20295076]]
[[6.15982623]]
[[6.11909498]]
[[6.08060107]]
[[6.04419668]]
[[6.00974199]]
[[5.97710495]]
[[5.94616105]]
[[5.91679307]]
[[5.88889084]]
[[5.86235093]]
[[5.8370764]]
[[5.81297649]]
[[5.78996634]]
[[5.7679667]]
[[5.74690364]]
[[5.72670827]]
[[5.70731646]]
[[5.68866856]]
[[5.67070916]]
[[5.65338681]]
[[5.63665379]]
[[5.62046587]]
[[5.60478209]]
[[5.58956454]]
[[5.57477816]]
[[5.56039053]]
[[5.54637172]]
[[5.5326941]]
[[5.51933216]]
[[5.50626238]]
[[5.49346307]]
[[5.48091426]]
[[5.46859754]]
[[5.45649597]]
[[5.44459395]]
[[5.43287712]]
[[5.42133228]]
[[5.4099473]]
[[5.398711]]
[[5.38761311]]
[[5.37664422]]
[[5.36579564]]
[[5.35505942]]
[[5.34442822]]
[[5.33389533]]
[[5.32345457]]
[[5.31310026]]
[[5.30282718]]
[[

[[1.51244903]]
[[1.50802567]]
[[1.50360827]]
[[1.49919684]]
[[1.4947914]]
[[1.49039196]]
[[1.48599855]]
[[1.48161116]]
[[1.47722982]]
[[1.47285455]]
[[1.46848535]]
[[1.46412225]]
[[1.45976525]]
[[1.45541438]]
[[1.45106964]]
[[1.44673105]]
[[1.44239863]]
[[1.43807239]]
[[1.43375235]]
[[1.42943851]]
[[1.42513091]]
[[1.42082954]]
[[1.41653442]]
[[1.41224558]]
[[1.40796301]]
[[1.40368675]]
[[1.3994168]]
[[1.39515317]]
[[1.39089589]]
[[1.38664496]]
[[1.38240041]]
[[1.37816224]]
[[1.37393046]]
[[1.36970511]]
[[1.36548618]]
[[1.36127369]]
[[1.35706767]]
[[1.35286811]]
[[1.34867504]]
[[1.34448847]]
[[1.34030841]]
[[1.33613488]]
[[1.33196789]]
[[1.32780746]]
[[1.3236536]]
[[1.31950633]]
[[1.31536565]]
[[1.31123159]]
[[1.30710415]]
[[1.30298335]]
[[1.29886921]]
[[1.29476173]]
[[1.29066093]]
[[1.28656684]]
[[1.28247945]]
[[1.27839878]]
[[1.27432484]]
[[1.27025766]]
[[1.26619724]]
[[1.2621436]]
[[1.25809675]]
[[1.2540567]]
[[1.25002346]]
[[1.24599706]]
[[1.24197749]]
[[1.23796479]]
[[1.23395895]]


[[0.05403206]]
[[0.05371327]]
[[0.05339625]]
[[0.05308098]]
[[0.05276746]]
[[0.05245567]]
[[0.05214562]]
[[0.05183728]]
[[0.05153066]]
[[0.05122574]]
[[0.05092251]]
[[0.05062097]]
[[0.0503211]]
[[0.05002291]]
[[0.04972638]]
[[0.0494315]]
[[0.04913826]]
[[0.04884666]]
[[0.04855669]]
[[0.04826834]]
[[0.0479816]]
[[0.04769647]]
[[0.04741293]]
[[0.04713097]]
[[0.0468506]]
[[0.0465718]]
[[0.04629456]]
[[0.04601887]]
[[0.04574474]]
[[0.04547214]]
[[0.04520108]]
[[0.04493154]]
[[0.04466351]]
[[0.044397]]
[[0.04413198]]
[[0.04386846]]
[[0.04360643]]
[[0.04334587]]
[[0.04308679]]
[[0.04282916]]
[[0.04257299]]
[[0.04231828]]
[[0.042065]]
[[0.04181315]]
[[0.04156273]]
[[0.04131373]]
[[0.04106614]]
[[0.04081995]]
[[0.04057516]]
[[0.04033177]]
[[0.04008975]]
[[0.03984911]]
[[0.03960983]]
[[0.03937192]]
[[0.03913536]]
[[0.03890015]]
[[0.03866627]]
[[0.03843373]]
[[0.03820252]]
[[0.03797262]]
[[0.03774404]]
[[0.03751676]]
[[0.03729078]]
[[0.0370661]]
[[0.03684269]]
[[0.03662057]]
[[0.03639972]]
[[0.0

[[0.00020913]]
[[0.00020775]]
[[0.00020637]]
[[0.00020501]]
[[0.00020366]]
[[0.00020232]]
[[0.00020098]]
[[0.00019966]]
[[0.00019834]]
[[0.00019703]]
[[0.00019573]]
[[0.00019444]]
[[0.00019315]]
[[0.00019188]]
[[0.00019061]]
[[0.00018936]]
[[0.00018811]]
[[0.00018686]]
[[0.00018563]]
[[0.00018441]]
[[0.00018319]]
[[0.00018198]]
[[0.00018078]]
[[0.00017958]]
[[0.0001784]]
[[0.00017722]]
[[0.00017605]]
[[0.00017489]]
[[0.00017373]]
[[0.00017259]]
[[0.00017145]]
[[0.00017032]]
[[0.00016919]]
[[0.00016807]]
[[0.00016697]]
[[0.00016586]]
[[0.00016477]]
[[0.00016368]]
[[0.0001626]]
[[0.00016153]]
[[0.00016046]]
[[0.0001594]]
[[0.00015835]]
[[0.0001573]]
[[0.00015626]]
[[0.00015523]]
[[0.0001542]]
[[0.00015319]]
[[0.00015217]]
[[0.00015117]]
[[0.00015017]]
[[0.00014918]]
[[0.00014819]]
[[0.00014722]]
[[0.00014624]]
[[0.00014528]]
[[0.00014432]]
[[0.00014336]]
[[0.00014242]]
[[0.00014148]]
[[0.00014054]]
[[0.00013961]]
[[0.00013869]]
[[0.00013777]]
[[0.00013686]]
[[0.00013596]]
[[0.00013506]]
